# Usuwanie błędów 
Korzystając z rozwiązań z poprzedniego wykładu oraz pliku **ceny_produktow_renamed.csv**, zmodyfikuj `loc`, tak aby poprawił istniejące błędy w zbiorze danych:

1. W kolumnie **date** pojawiły się dane z roku 1888 - '1888-0', zmień tę wartość na 1999-1,
1. W kolumnie **date** pojawił się dane z roku 2099 - '2099-13', zmień tę wartość na 2019-1,
1. W kolumnie **product_types** pojawił się błąd ortograficzny - popraw go. Liczba sztuk powinna wynosić '10szt.`. Sprawdź poprawność wykonania zadania.
1. Przekonwertuj wartości podane w `EUR` na `zł` używając kursu 4.15 używając `loc`.
1. Odfiltuj ze zbioru te wiersze, gdzie cena za produkt wynosi 3000.
Wskazówka: Zamiast pisać `loc` dwa razy, zrób go najpierw, odpytując najpierw dane o wiersze, gdzie **currency** = `EUR` i zapisz do zmiennej.

>Pamiętaj, że `loc` modyfikuje bezpowrotnie dane.

In [18]:
import pandas as pd
df = pd.read_csv(
  r'C:\Users\damia\Desktop\coderslab - Data scientist\3. Pandas\Zjazd_3_-_pliki_do_zadań\Zjazd 3 - pliki do zadań\01_Dane\ceny_produktow_renamed.csv',
  sep=';',
  encoding='UTF-8',
  decimal='.'
)
df_clean = df.copy()

In [19]:
def overwrite_df_values(df, col, old_val, new_val):
    df.loc[(df[col] == old_val), col] = new_val
    if df.loc[df[col] == old_val].shape[0] > 0:
        raise Exception("Couldn't replace all the values")

In [20]:
df_clean['product_types'].unique()

array([nan, 'ogórki konserwowe całe 0,9l - za 1szt.',
       'jaja kóże śfierze - za 666szt.',
       'koncentrat pomidorowy 30% - za 1kg',
       'mieszanka mrożona marchew i groszek - za 1kg',
       'cukier biały z buraków cukrowych, workowany - za 1kg',
       'sok jabłkowy, w opakowaniu kartonowym - za 1l',
       'sól warzona biała workowana - za 1kg',
       'czekolada naturalna pełna bez dodatków - za 1kg'], dtype=object)

In [21]:
# 1 1888-0 -> 1999-1 (date)
df_clean.loc[(df_clean['date'] == '1888-0'), 'date'] = '1999-1'
print(f"Liczba wpisów z datą 1888-0: {df_clean.loc[(df_clean['date'] == '1888-0')].shape[0]}")

# 2 2099-13 -> 2019-1 (date)
overwrite_df_values(df_clean, 'date', '2099-13', '2019-1')
print(f"Liczba wpisów z datą 2099-13: {df_clean.loc[(df_clean['date'] == '2099-13')].shape[0]}")

# 3 'jaja kóże śfierze - za 666szt.' -> 'jaja kurze świeże - za 10szt.' (product_types)
df_clean.loc[(df_clean['product_types'] == 'jaja kóże śfierze - za 666szt.'), 'product_types'] = 'jaja kurze świeże - za 10szt.'

# overwrite_df_values(
#     df_clean, 
#     'product_types', 
#     'jaja kóże śfierze - za 666szt.',
#     'jaja kurze świeże - za 10szt.',
# )

Liczba wpisów z datą 1888-0: 0
Liczba wpisów z datą 2099-13: 0


In [22]:
df_clean.loc[(df_clean['product_types'] == 'jaja kóże śfierze - za 666szt.'), 'product_types'] = 'jaja kurze świeże - za 10szt.'

In [23]:
df_clean.loc[(df_clean['product_types'] == 'jaja kóże śfierze - za 666szt.')]

,province,product_types,currency,product_group_id,product_line,value,date


In [24]:
df_clean.loc[df_clean['date'] == '2099-13']

,province,product_types,currency,product_group_id,product_line,value,date


In [25]:
# 4 EUR -> zł (currency, x4.15)
eur_index = df_clean['currency'] == 'EUR'

In [26]:
df_clean.loc[26]

province                                            DOLNOŚLĄSKIE
product_types       mieszanka mrożona marchew i groszek - za 1kg
currency                                                     EUR
product_group_id                                               1
product_line                                                 NaN
value                                                    0.66988
date                                                      2005-7
Name: 26, dtype: object

In [27]:
df_clean.loc[eur_index].head(1)

,province,product_types,currency,product_group_id,product_line,value,date
26,DOLNOŚLĄSKIE,mieszanka mrożona marchew i groszek - za 1kg,EUR,1,NaN,0.66988,2005-7


In [28]:
# uwaga, jesli nie skorzystamy ze zmiennej eur_index
# to musimy koniecznie najpiewr przemnozyc przez 4.15, a dopiero pozniej zamienic EUR -> zł
df_clean.loc[eur_index, 'value'] = df_clean['value']*4.15
df_clean.loc[eur_index, 'currency'] = 'zł'
# -> wyskakuje warning poniżej
# df_clean.loc[eur_index, ['currency', 'value']] = 'zł', df_clean['value']*4.15
df_clean.loc[eur_index].head(1)

,province,product_types,currency,product_group_id,product_line,value,date
26,DOLNOŚLĄSKIE,mieszanka mrożona marchew i groszek - za 1kg,zł,1,NaN,2.78,2005-7


In [17]:
# 5 value==3000 -> gone
df_clean = df_clean.loc[(df_clean['value'] != 3000) & (df_clean['value'] != 0)]

df_clean.to_csv('produkty_cleaned.csv', sep=';', decimal='.', encoding='utf-8', index=False)